In [1]:
import json
with open('/Users/corte/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['Client-ID', 'API Key'])

In [2]:
# import the YelpAPI Class
from yelpapi import YelpAPI
# Create an instance with your key
yelp_api = YelpAPI(login['API Key'], timeout_s=5.0)
yelp_api

In [3]:
help(yelp_api.search_query)

Help on method search_query in module yelpapi.yelpapi:

search_query(**kwargs) method of yelpapi.yelpapi.YelpAPI instance
    Query the Yelp Search API.
    
    documentation: https://www.yelp.com/developers/documentation/v3/business_search
    
    required parameters:
        * one of either:
            * location - text specifying a location to search for
            * latitude and longitude



In [4]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [5]:
# set our API call parameters 
LOCATION = 'Seattle, WA'
TERM = 'Vegetarian'

In [6]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/final_results_seattle_vegetarian.json"
JSON_FILE

'Data/final_results_seattle_vegetarian.json'

In [7]:
# Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/final_results_seattle_vegetarian.json not found. Saving empty list to file.


In [8]:
# Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [9]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [10]:
# How many results total?
total_results = results['total']
total_results

4400

In [11]:
# How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [12]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

220

In [13]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [14]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/220 [00:00<?, ?it/s]

In [16]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [17]:
# Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages


[!] Data/final_results_seattle_vegetarian.json already exists. Deleting previous file...
[i] Data/final_results_seattle_vegetarian.json not found. Saving empty list to new file.
- 0 previous results found.


220

In [18]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)


  0%|          | 0/220 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [20]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,djjEdAZkY4eyzI3_8TBLVA,plum-bistro-seattle-2,Plum Bistro,https://s3-media3.fl.yelpcdn.com/bphoto/yMrYwh...,False,https://www.yelp.com/biz/plum-bistro-seattle-2...,1880,"[{'alias': 'vegan', 'title': 'Vegan'}]",4.0,"{'latitude': 47.6137675, 'longitude': -122.317...","[pickup, delivery]",$$,"{'address1': '1429 12th Ave', 'address2': '', ...",+12068385333,(206) 838-5333,1888.191530
1,mOybprI4rEcRQUxBqAhupw,harvest-beat-seattle,Harvest Beat,https://s3-media2.fl.yelpcdn.com/bphoto/QPuRj9...,False,https://www.yelp.com/biz/harvest-beat-seattle?...,255,"[{'alias': 'vegetarian', 'title': 'Vegetarian'...",4.5,"{'latitude': 47.66115, 'longitude': -122.33678}",[],$$$,"{'address1': '1711 N 45th St', 'address2': '',...",+12065471348,(206) 547-1348,3976.167517
2,0jZIOA_9H3YOsbiyoyRWrA,seattle-lotus-vegetarian-restaurant-seattle,Seattle Lotus Vegetarian Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/GWvM2a...,False,https://www.yelp.com/biz/seattle-lotus-vegetar...,194,"[{'alias': 'vegetarian', 'title': 'Vegetarian'...",4.5,"{'latitude': 47.5088066528403, 'longitude': -1...","[pickup, delivery]",$$,"{'address1': '10439 16th Ave SW', 'address2': ...",+12064666345,(206) 466-6345,13054.220036
3,kh_RJCN-Xn7sP5tenN2WsA,ba-bar-green-seattle,Ba Bar Green,https://s3-media2.fl.yelpcdn.com/bphoto/8h7jos...,False,https://www.yelp.com/biz/ba-bar-green-seattle?...,31,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",4.5,"{'latitude': 47.62343, 'longitude': -122.33684}","[pickup, delivery]",NaN,"{'address1': '500 Terry Ave N', 'address2': ''...",+12065822131,(206) 582-2131,251.250077
4,9nJlVsm9GEz6DFhK5ckaYA,cafe-flora-seattle,Cafe Flora,https://s3-media1.fl.yelpcdn.com/bphoto/M33I7s...,False,https://www.yelp.com/biz/cafe-flora-seattle?ad...,1558,"[{'alias': 'vegetarian', 'title': 'Vegetarian'...",4.0,"{'latitude': 47.62397, 'longitude': -122.29521}",[delivery],$$,"{'address1': '2901 E Madison St', 'address2': ...",+12063259100,(206) 325-9100,3033.471253


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,_7i3cfZt8A0tBwyTsPpH4w,fiddlers-inn-pub-and-pizza-seattle,Fiddler's Inn Pub & Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/Ee3tn4...,False,https://www.yelp.com/biz/fiddlers-inn-pub-and-...,161,"[{'alias': 'pubs', 'title': 'Pubs'}, {'alias':...",4.0,"{'latitude': 47.6963958740234, 'longitude': -1...",[delivery],$$,"{'address1': '9219 35th Ave NE', 'address2': '...",+12065250752,(206) 525-0752,8567.291262
996,yHfENRcxUcq1UWtZrxMqtA,nacho-mama-s-fry-bread-cafe-seattle,Nacho Mama’s Fry Bread Cafe’,https://s3-media1.fl.yelpcdn.com/bphoto/9ICQSW...,False,https://www.yelp.com/biz/nacho-mama-s-fry-brea...,17,"[{'alias': 'breakfast_brunch', 'title': 'Break...",5.0,"{'latitude': 47.51866236481751, 'longitude': -...","[delivery, pickup]",NaN,"{'address1': '9418 Delridge Way S', 'address2'...",+12533291212,(253) 329-1212,11970.301412
997,Typ4u1aCYFBXaki_ylNTuQ,s-t-hooligans-seattle,S/T Hooligan's,https://s3-media2.fl.yelpcdn.com/bphoto/vfFWEy...,False,https://www.yelp.com/biz/s-t-hooligans-seattle...,431,"[{'alias': 'cajun', 'title': 'Cajun/Creole'}, ...",4.5,"{'latitude': 47.66023, 'longitude': -122.36568}","[delivery, pickup]",$$,"{'address1': '4354 Leary Way NW', 'address2': ...",+12066594754,(206) 659-4754,4482.383222
998,uakk24V_xpD-INF6kxc4rg,menya-musashi-tsukemen-and-ramen-seattle-6,Menya Musashi Tsukemen & Ramen,https://s3-media1.fl.yelpcdn.com/bphoto/VFS-6C...,False,https://www.yelp.com/biz/menya-musashi-tsukeme...,570,"[{'alias': 'ramen', 'title': 'Ramen'}, {'alias...",4.0,"{'latitude': 47.61443, 'longitude': -122.32423}","[pickup, delivery]",$$,"{'address1': '1510 Belmont Ave', 'address2': '...",+12062574289,(206) 257-4289,1498.765728
999,eoXUJKoAacpQ7F57xu8rNw,flavor-of-india-issaquah,Flavor of India,https://s3-media2.fl.yelpcdn.com/bphoto/bjHzYz...,False,https://www.yelp.com/biz/flavor-of-india-issaq...,25,"[{'alias': 'indpak', 'title': 'Indian'}, {'ali...",4.0,"{'latitude': 47.5435, 'longitude': -122.01664}","[delivery, pickup]",NaN,"{'address1': '984 NE Park Dr', 'address2': '',...",+14253694334,(425) 369-4334,25588.961885


In [21]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

0

There are no duplicates

In [22]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_seattle_vegetarian.csv', compression='gzip',index=False)